In [1]:
"""
Pre_processing of the fastq files

The following code has been written to split the fastq files into component files/

"""

def hamming_distance(string1,string2):
    """
    The following code is to measure the hamming distance between 2 reads
    Input: String 1 and string 2
    
    Output: Distance(count of mismatches between the 2 strings)
    """
    count=0
    for x,y  in zip(string1,string2):
        if x!=y:
            count+=1
    return count

def fastq_splitter_pairedend(R1file,R2file,Indexfile,BCmapfile,levdist,outdir):
    """
    The following function is to split the sequences into reapective fastq files by index
    """
    import time
    time0=time.time()
    import gzip
    #import BioModules
    from itertools import izip
    #from BioModules import revcomp
    import collections
    #import Levenshtein as Lev
    import io
    counts = collections.defaultdict(int)
    totalseqs=0
    assigned=0
    R1=gzip.open(R1file,'rb')
    R2=gzip.open(R2file,'rb')
    I1=gzip.open(Indexfile,'rb')
    mydict=[]
    BCs=[]
    counterlist=[]
    with open(BCmapfile,'r') as f:
        for line in f:
            line=line.strip().split('\t')
            BCs.append((line[0],line[1]))
            
    outfsR1 = dict([(ID, open(outdir+ID + "_R1.fastq", "w"))
                 for ID,seq in BCs])
    outfsR2 = dict([(ID, open(outdir+ID + "_R2.fastq", "w"))
                 for ID,seq in BCs])
    iterator=enumerate(izip(R1,R2,I1))
    
    for i, (line1,line2,line3) in iterator:
        
        totalseqs+=1
        R1_ID,R2_ID,I1_ID=line1,line2.rstrip(),line3
        seqs=iterator.next()
        seq1,seq2,index=seqs[1][0],seqs[1][1].strip(),seqs[1][2][1:]
        spacer=iterator.next()
        quals=iterator.next()
        q1,q2,qI=quals[1][0],quals[1][1][::-1],quals[1][2]
        
        for ID,BC in BCs: 
        #Defines the acceptable deviations of the index read from the mapping barcodes
        # and looks for kmers in the sequences. Can do various different processing steps here
            if hamming_distance(index,BC.strip())<=levdist:
                group=ID
                assigned+=1
                #trim and concatenate reads 1&2 and write to file, can alter the behavior here as desired
                outfsR1[group].write(R1_ID.strip()+'\n')
                outfsR1[group].write(seq1.strip()+'\n')
                outfsR1[group].write('+'+'\n')
                outfsR1[group].write(q1.strip()+'\n')
                outfsR2[group].write(R2_ID.strip()+'\n')
                outfsR2[group].write(seq2.strip()+'\n')
                outfsR2[group].write('+'+'\n')
                outfsR2[group].write(q2.strip()+'\n')
                counts[group]+=1
                counterlist.append([ID,BC,index])
                break   
    for i in outfsR1.values():
        i.close()
    for i in outfsR2.values():
        i.close()
    print time.time()-time0
    #return counterlist

In [2]:
import os
os.chdir("/home/thegillgroup/Alaksh/MySeq_Data_Analysis_Pipeline/")
R1file = "Undetermined_S0_L001_R1_001.fastq.gz"
R2file = "Undetermined_S0_L001_R2_001.fastq.gz"
Indexfile = "Undetermined_S0_L001_I1_001.fastq.gz"
BCmapfile = "map.txt"
levdist = 4
outdir = "BC_Split_Files/"
fastq_splitter_pairedend(R1file,R2file,Indexfile,BCmapfile,levdist,outdir)

OSError: [Errno 2] No such file or directory: '/home/thegillgroup/Alaksh/MySeq_Data_Analysis_Pipeline/'

In [25]:
"""
The following code is to merge the different files

The implmentation is done using pandaseq

Here are the default parameters:


The following code is written to merge the fastq files
"""
import subprocess
import os
os.chdir("/home/thegillgroup/Alaksh/MySeq_Data_Analysis_Pipeline/BC_Split_Files/")
split_files = os.listdir("/home/thegillgroup/Alaksh/MySeq_Data_Analysis_Pipeline/BC_Split_Files/")
list_barcodes = []
for files in split_files:
    list_barcodes.append(files[0:-9])
list_barcodes = list(set(list_barcodes))
list_barcodes.remove('')

for barcode in list_barcodes:
    for_read_file = barcode + "_R1.fastq" #forward read goes here
    rev_read_file = barcode + "_R2.fastq" #reverse read goes here
    output_file = barcode + "_output.fasta" #Output file goes here
    p = subprocess.call(["pandaseq","-f",for_read_file,"-r",rev_read_file,"-w",output_file])
    if p == 0:
        print("Command was executed succesfully")
    else:
        print("Command didn't run")

Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully
Command was executed succesfully


In [5]:
"""
The following code is to read fasta files:

Input: filename
output: 2 lists one with sequences and other with ids
"""
import os
os.chdir("/Users/andrewmorgenthaler/Google Drive/MCDB/Courses/Bioinformatics/project/")
def readFasta(filename):
    seq_id_list = []
    sequences = []
    with open(filename) as fh:
        while True:
            seq_id = fh.readline().rstrip()[1:]
            seq = fh.readline().rstrip()
            if len(seq) == 0:
                break
            seq_id_list.append(seq_id)
            sequences.append(seq)
    return seq_id_list,sequences
seq_id_list,sequences = readFasta("CREATElibrary_folAsaturation.fasta")

In [3]:
import subprocess
import os
os.chdir("/Users/andrewmorgenthaler/Google Drive/MCDB/Courses/Bioinformatics/project/")
split_files = os.listdir("/Users/andrewmorgenthaler/Google Drive/MCDB/Courses/Bioinformatics/project/data/")
file_list = []


def usearch_function(file_name,output_filename):
    userfields = "query+target+id+alnlen+mism+opens+qlo+qhi+tlo+thi+evalue+bits+qrow+trow+qrowdots+tstrand+qstrand"
    p = subprocess.call(["/Users/andrewmorgenthaler/Google Drive/MCDB/Courses/Bioinformatics/project/usearch9.2.64","-usearch_global",file_name,"-db","CREATElibrary_folAsaturation.fasta","-id","0.8",\
                        "-strand","both","-userout",output_filename,"-userfields",\
                        userfields])
    return p

for files in split_files:
    input_file = 'data/'+files
    output_file = 'Usearch_Output/'+files[0:-13]+"_usearch.txt"
    print output_file
    p = usearch_function(input_file,output_file)
    if p == 0:
        print("Usearch completed successfuly for -" + input_file)
    else:
        print("Usearch not done for -" + input_file)
    

Usearch_Output/BC01_folA_s1_T0_usearch.txt
Usearch completed successfuly for -data/BC01_folA_s1_T0_merged.fastq
Usearch_Output/BC04_folA_s1_TMP_usearch.txt
Usearch completed successfuly for -data/BC04_folA_s1_TMP_merged.fastq
Usearch_Output/BC05_folA_s2_T0_usearch.txt
Usearch completed successfuly for -data/BC05_folA_s2_T0_merged.fastq
Usearch_Output/BC08_folA_s2_TMP_usearch.txt
Usearch completed successfuly for -data/BC08_folA_s2_TMP_merged.fastq
